In [ ]:
import pandas as pd
chemin_fichier = '/content/scopus.csv'
df_global = pd.read_csv(chemin_fichier)
df_global.head()


In [ ]:
print(df_global.columns.tolist())

In [ ]:
print(df_global['Affiliations'])

In [ ]:
%pip install -qU langchain-groq


In [ ]:
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field

class EntityExtraction(BaseModel):
    university: str = Field(description="Nom de l'université")
    faculty: str = Field(description="Nom de la faculté")
    city: str = Field(description="Nom de la ville")
    country: str = Field(description="Nom du pays")

api_key = ""
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)

query = "University of Sidi Mohamed Ben Abdellah, Faculty of Science and Technology, Laboratory of Functional Ecology and Environmental Engineering, Fez, Morocco"

messages = [
    (
        "system",
        "You are a helpful assistant that extracts entities such as school (école), faculty (faculté), city (ville), and country (pays) from a sentence. "
        "You should output them in the following format: {'school': '<school_name>', 'faculty': '<faculty_name>', 'city': '<city_name>', 'country': '<country_name>', 'university': '<university_name>'}. "
        "If the sentence includes an acronym (e.g., INPT for Institut National des Postes et Télécommunications), "
        "you should expand it to the full name. "
        "Recognize abbreviations like INPT, ENSA, FST, and others and expand them to their full names. "
        "If the school is associated with a well-known university in the given city, output the university's name. "
        "For example, if the school is ENS in Tetouan, the university should be Université Abdelmalek Essaadi, Tetouan. "
        "If there is no direct association with a university in the city, state 'There is no specific university mentioned for <school_name>, <city>.'."
    ),
    ("human", query),
]

ai_msg = llm.invoke(messages)

print(ai_msg.content)

try:
    response_data = ai_msg.content

    entities = EntityExtraction.parse_raw(response_data)
    print(entities)
except Exception as e:
    print("Erreur lors du parsing de la réponse:", e)


In [ ]:
df_affiliations_split = df_global['Affiliations'].str.split(';', expand=True)

df_affiliations_split.head(2)

In [ ]:
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import json

class EntityExtraction(BaseModel):
    author: str = Field(description="Nom de l'auteur")
    university: str = Field(description="Nom de l'université")
    year: int = Field(description="Année de publication")

api_key = ""
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=200,
    timeout=None,
    max_retries=2,
    api_key=api_key
)

universities = [
    "tkhbi9a university"
    "University of abdelmalek saadi "
    "University of Sidi Mohamed Ben Abdellah, Faculty of Science and Technology, Laboratory of Functional Ecology and Environmental Engineering, Fez, Morocco",
    "University of Sidi Mohamed Ben Abdellah, Faculty of Science and Technology, Laboratory of Condensed Matter Chemistry, Fez, Morocco",
    "Mohammed 1st University, Nador Multidisciplinary Faculty (FPN), OLMANBGPE, Oujda, Nador, Morocco"

]

def extract_affiliations_from_text(query):
    messages = [
        (
            "system",
            "You are an assistant that extracts author affiliations with universities. You are given the name of an author and the year of publication. "
            "Provide the author's affiliation with the university in the following format: {'author': '<author_name>', 'university': '<university_name>', 'year': <year>}. "
            "If there is no affiliation, indicate 'No affiliation found for <author_name> in <year>'."
        ),
        ("human", query),
    ]


    ai_msg = llm.invoke(messages)

    try:

        response_data = ai_msg.content
        print("Réponse de Groq:", response_data)


        if response_data:

            entities = json.loads(response_data)

            if isinstance(entities, dict) and 'author' in entities and 'university' in entities and 'year' in entities:
                return entities
            else:
                return {'error': 'Réponse mal formatée', 'author': query.split(' ')[3]}
        else:
            return {'error': 'Aucune réponse reçue'}

    except Exception as e:
        print(f"Erreur lors du parsing de la réponse : {e}")
        return {'error': f"Erreur de parsing: {e}"}

authors = ["Chater Oumaima", "El Ghadraoui Lahsen", "Harrach Ahmed", "Aazza Smail"]
results = {}

for author in authors:

    query = f"Quelle est l'affiliation de {author} en 2023 ? Il est associé aux universités suivantes : {', '.join(universities)}"

    affiliation = extract_affiliations_from_text(query)
    results[author] = affiliation

for author, affiliation in results.items():
    print(f"Résultats pour {author}: {affiliation}")


In [ ]:
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import json

class EntityExtraction(BaseModel):
    author: str = Field(description="Nom de l'auteur")
    university: str = Field(description="Nom de l'université")
    year: int = Field(description="Année de publication")

api_key = ""
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=200,
    timeout=None,
    max_retries=2,
    api_key=api_key
)

universities = [

    "University of abdelmalek saadi",
    "University of Sidi Mohamed Ben Abdellah, Faculty of Science and Technology, Laboratory of Functional Ecology and Environmental Engineering, Fez, Morocco",
    "University of Sidi Mohamed Ben Abdellah, Faculty of Science and Technology, Laboratory of Condensed Matter Chemistry, Fez, Morocco",
    "Mohammed 1st University, Nador Multidisciplinary Faculty (FPN), OLMANBGPE, Oujda, Nador, Morocco"
]

def extract_affiliations_from_text(query):
    messages = [
        (
            "system",
            "You are an assistant that extracts author affiliations with universities. You are given the name of an author and the year of publication. "
            "Provide the author's affiliation with the university in the following format: {'author': '<author_name>', 'university': '<university_name>', 'year': <year>}. "
            "If there is no affiliation, indicate 'No affiliation found for <author_name> in <year>'."
        ),
        ("human", query),
    ]

    ai_msg = llm.invoke(messages)

    try:
        response_data = ai_msg.content
        print("Réponse de Groq:", response_data)

        if response_data:
            entities = json.loads(response_data)
            if isinstance(entities, dict) and 'author' in entities and 'university' in entities and 'year' in entities:
                return entities
            else:
                return {'error': 'Réponse mal formatée', 'author': query.split(' ')[3]}
        else:
            return {'error': 'Aucune réponse reçue'}

    except Exception as e:
        print(f"Erreur lors du parsing de la réponse : {e}")
        return {'error': f"Erreur de parsing: {e}"}

authors = ["Chater Oumaima", "El Ghadraoui Lahsen", "Harrach Ahmed", "Aazza Smail"]
results = {}

for author in authors:
    query = f"Quelle est l'affiliation de {author} en 2023 ? Il est associé aux universités suivantes : {', '.join(universities)}"
    affiliation = extract_affiliations_from_text(query)
    results[author] = affiliation

output_file = "affiliations_results.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

print(f"Les résultats ont été sauvegardés dans le fichier : {output_file}")


In [ ]:
import pandas as pd
chemin_fichier = '/content/morocco_affiliations.json'
df_json = pd.read_json(chemin_fichier)
print("Contenu du fichier JSON sous forme de DataFrame :")
df_json.head()


In [ ]:
import pandas as pd
chemin_fichier = '/content/morocco_affiliations.json'
df_json = pd.read_json(chemin_fichier)

if 'affiliation' in df_json.columns:

    df_exploded = df_json['affiliation'].explode().reset_index(drop=True)

    affiliation_df = pd.json_normalize(df_exploded)

    affiliation_df = affiliation_df[['affilname', 'affiliation-city', 'affiliation-country']]

    morocco_affiliations = affiliation_df[affiliation_df['affiliation-country'] == 'Morocco']

    print("Affiliations dans le pays 'Morocco' :")
    print(morocco_affiliations)
else:
    print("La colonne 'affiliation' n'existe pas dans ce fichier JSON.")


In [ ]:
import json
import pandas as pd
import uuid

with open("morocco_affiliations.json", "r", encoding="utf-8") as file:
    data = json.load(file)

authors_data = []
authors_mapping = {}

affiliations_data = []
affiliation_mapping = {}

journals_data = []
facts_data = []

for entry in data:

    journal_id = str(uuid.uuid4())
    journals_data.append({
        "journal_id": journal_id,
        "title": entry.get("dc:title"),
        "doi": entry.get("prism:doi"),
        "url": entry.get("prism:url"),
        "publication_name": entry.get("prism:publicationName"),
        "issn": entry.get("prism:issn"),
        "eissn": entry.get("prism:eIssn"),
        "volume": entry.get("prism:volume"),
        "issue": entry.get("prism:issueIdentifier"),
        "page_range": entry.get("prism:pageRange"),
        "cover_date": entry.get("prism:coverDate"),
        "cited_by": entry.get("citedby-count")
    })

    author_name = entry.get("dc:creator", "")
    if author_name not in authors_mapping:
        author_id = str(uuid.uuid4())
        authors_mapping[author_name] = author_id
        authors_data.append({
            "author_id": author_id,
            "author_name": author_name
        })
    else:
        author_id = authors_mapping[author_name]

    affiliations = entry.get("affiliation", [])
    if isinstance(affiliations, list):
        for affil in affiliations:
            affil_name = affil.get("affilname", "").strip()
            affil_city = affil.get("affiliation-city", "")
            affil_country = affil.get("affiliation-country", "")


            affil_key = f"{affil_name}_{affil_city}_{affil_country}"
            if affil_key not in affiliation_mapping:
                affiliation_id = str(uuid.uuid4())
                affiliation_mapping[affil_key] = affiliation_id
                affiliations_data.append({
                    "affiliation_id": affiliation_id,
                    "affilname": affil_name,
                    "city": affil_city,
                    "country": affil_country,
                    "author_id": author_id
                })
            else:
                affiliation_id = affiliation_mapping[affil_key]

            facts_data.append({
                "journal_id": journal_id,
                "author_id": author_id,
                "affiliation_id": affiliation_id
            })

authors_df = pd.DataFrame(authors_data)
affiliations_df = pd.DataFrame(affiliations_data)
journals_df = pd.DataFrame(journals_data)
facts_df = pd.DataFrame(facts_data)

facts_df


In [ ]:
import json
import pandas as pd
import uuid

with open("morocco_affiliations.json", "r", encoding="utf-8") as file:
    data = json.load(file)

authors_data = []
authors_mapping = {}

affiliations_data = []
affiliation_mapping = {}

journals_data = []
facts_data = []

for entry in data:

    journal_id = str(uuid.uuid4())
    journal_data = {
        "journal_id": journal_id,
        "url": entry.get("prism:url"),
        "identifier": entry.get("dc:identifier"),
        "eid": entry.get("eid"),
        "title": entry.get("dc:title"),
        "creator": entry.get("dc:creator"),
        "publication_name": entry.get("prism:publicationName"),
        "issn": entry.get("prism:issn"),
        "eissn": entry.get("prism:eIssn"),
        "volume": entry.get("prism:volume"),
        "issue": entry.get("prism:issueIdentifier"),
        "page_range": entry.get("prism:pageRange"),
        "cover_date": entry.get("prism:coverDate"),
        "cited_by": entry.get("citedby-count"),
        "doi": entry.get("prism:doi")
    }

    journals_data.append(journal_data)

    author_name = entry.get("dc:creator", "")
    if author_name not in authors_mapping:
        author_id = str(uuid.uuid4())
        authors_mapping[author_name] = author_id
        authors_data.append({
            "author_id": author_id,
            "author_name": author_name
        })
    else:
        author_id = authors_mapping[author_name]

    affiliations = entry.get("affiliation", [])
    if isinstance(affiliations, list):
        for affil in affiliations:
            affil_name = affil.get("affilname", "").strip()
            affil_city = affil.get("affiliation-city", "")
            affil_country = affil.get("affiliation-country", "")

            affil_key = f"{affil_name}_{affil_city}_{affil_country}"
            if affil_key not in affiliation_mapping:
                affiliation_id = str(uuid.uuid4())
                affiliation_mapping[affil_key] = affiliation_id
                affiliations_data.append({
                    "affiliation_id": affiliation_id,
                    "affilname": affil_name,
                    "city": affil_city,
                    "country": affil_country,
                    "author_id": author_id
                })
            else:
                affiliation_id = affiliation_mapping[affil_key]


            facts_data.append({
                "journal_id": journal_id,
                "author_id": author_id,
                "affiliation_id": affiliation_id,
                "journal_url": journal_data["url"],
                "journal_identifier": journal_data["identifier"],
                "journal_title": journal_data["title"],
                "journal_doi": journal_data["doi"],
                "journal_cited_by": journal_data["cited_by"]
            })

authors_df = pd.DataFrame(authors_data)
affiliations_df = pd.DataFrame(affiliations_data)
journals_df = pd.DataFrame(journals_data)
facts_df = pd.DataFrame(facts_data)

In [ ]:
journals_df

In [ ]:

journals_df["identifier"] = journals_df["identifier"].str.replace("SCOPUS_ID:", "", regex=False)
journals_df.head()


In [ ]:
authors_df

In [ ]:
affiliations_df

In [ ]:
facts_df

In [ ]:
print("Author ID (première ligne de authors_df) :", authors_df.iloc[0]["author_id"])
print("Author ID (première ligne de affiliations_df) :", affiliations_df.iloc[0]["author_id"])
print("Affiliation ID (première ligne de affiliations_df) :", affiliations_df.iloc[0]["affiliation_id"])


In [ ]:

facts_df.to_csv("facts.csv", index=False, encoding="utf-8")
affiliations_df.to_csv("affiliations.csv", index=False, encoding="utf-8")
authors_df.to_csv("authors.csv", index=False, encoding="utf-8")
journals_df.to_csv("journals.csv", index=False, encoding="utf-8")

print("Les fichiers CSV ont été générés avec succès : facts.csv, affiliations.csv, authors.csv, journals.csv")


In [ ]:
import pandas as pd

def find_all_differences(base, compare):
    results = []
    matched_lines = []

    for phrase1 in base:
        row_results = []
        matched_line = None
        for i, phrase2 in enumerate(compare):
            if phrase1 == phrase2:
                matched_line = phrase2
                break
            elif set(phrase1.split(', ')).issubset(set(phrase2.split(', '))):
                matched_line = phrase2
                break

        if matched_line:
            diff = [word for word in matched_line.split(', ') if word not in phrase1.split(', ')]
            if diff:
                row_results.append(', '.join(diff))
            else:
                row_results.append("")
        else:
            row_results.append("")


        matched_lines.append(matched_line if matched_line else "")

        results.append(', '.join(row_results) if row_results else "")

    return results, matched_lines


differences, matched_lines = find_all_differences(morocco_affiliations['affiliation_combined'], df_global['Affiliations'])

df_result = pd.DataFrame({
    "Affiliation combinée (Morocco)": morocco_affiliations['affiliation_combined'],
    "Différences trouvées dans df_global": differences,
    "Ligne correspondante dans df_global": matched_lines
})

pd.set_option('display.max_colwidth', None)

df_result.head()


In [ ]:
df_result = pd.DataFrame({
    "Affiliation combinée (Morocco)": morocco_affiliations['affiliation_combined'],
    "Différences trouvées dans df_global": differences,
    "Ligne correspondante dans df_global": matched_lines
})